<a href="https://colab.research.google.com/github/farsilvar/Image-CNN-FL/blob/main/createFigure_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %%
import numpy as np
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import image
from PIL import Image
import pandas as pd
import os
import shutil
import time
from tensorflow import keras as ks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math
from sklearn import preprocessing

In [ ]:
!pip3 install -U git+https://github.com/PYFTS/pyFTS

  Cloning https://github.com/PYFTS/pyFTS to /tmp/pip-req-build-m5hm9rt4
  Running command git clone --filter=blob:none --quiet https://github.com/PYFTS/pyFTS /tmp/pip-req-build-m5hm9rt4
  Resolved https://github.com/PYFTS/pyFTS to commit 4e0ee7cdd9fc9a8ac9170c0c3496536ef0378cb7
  Preparing metadata (setup.py) ... done
  Created wheel for pyFTS: filename=pyFTS-1.7-py3-none-any.whl size=219432 sha256=26ffbe3d07aaac910c717706d98e051de36e0e3a9b36659e9439bc6963fbeced
  Stored in directory: /tmp/pip-ephem-wheel-cache-qgp1lz25/wheels/7f/25/75/dd8f713bf48592da71e0441c7abdc1f5aa4d9645a6b7ee99d4
Successfully built pyFTS


In [ ]:
# Pyfts
import pandas as pd
import numpy as np
from pyFTS.partitioners import CMeans, Grid, FCM, Huarng, Entropy, Util as pUtil
from pyFTS.common import Membership as mf

In [ ]:
# from pandas_datareader import data

# %%
class TimeSeries:

    def __init__(self, rawData):
        self.rawData = rawData

        separateData = self.rawData.split(':')
        self.hasNA = 0
        try:
            self.data = np.array([float(item) for item in separateData[0].split(',')])
            self.label = int(separateData[1])
        except ValueError:
            self.hasNA = 1


    def fuzzycmeans(self):
        fcm = FCM(n_clusters=10)
        fcm.fit(np.array(self.data, ndmin = 2).T)
        xp = fcm.soft_predict(np.array(self.data, ndmin = 2).T)
        xpMax = np.max(xp, axis = 1)
        cluster = np.zeros(trainTS[i].data.shape[0])
        for n in range(self.data.shape[0]):
            cluster[n] = np.where(xp[n] == xpMax[n])[0]

        return np.array(cluster, dtype = int)

    def triangular(self,x, a, b, c):
        return max( min( (x-a)/(b-a), (c-x)/(c-b) ), 0 )

    def fts(self):
        maxValue = 1.2*np.max(self.data)
        minValue = 0.8*np.min(self.data)

        nGroups = 10

        A = np.zeros(nGroups+2)
        A[0] = minValue
        A[-1] = maxValue
        for i in range(1,nGroups+1):
            A[i] = A[i-1] + (maxValue - minValue)/(nGroups+1)

        fuzzy = np.zeros((self.data.shape[0], nGroups), dtype = float)
        j = 0
        for dado in self.data:
            fuzzyCalc = 0
            for u in range(1,A.shape[0]-1):
                calc = self.triangular(dado, A[u-1], A[u], A[u+1])
                fuzzy[j, u-1] = calc
            j = j+1

        return fuzzy

    def wpyfts(self, partType):

        nGroups = 10
        if partType == 'Grid':
            part = Grid.GridPartitioner(data = self.data, npart = 10)
        elif partType == 'Entropy':
            part = Entropy.EntropyPartitioner(data = self.data, npart = 10)
        elif partType == 'CMeans':
            part = CMeans.CMeansPartitioner(data = self.data, npart = 10)

        verticesArray = np.zeros((nGroups,3), dtype = float)
        i = 0

        for key, value in part.sets.items():
            verticesStr = str(value)[str(value).find('[')+1:str(value).find(']')]
            vertices = verticesStr.split(',')
            verticesArray[i] = np.array(vertices, dtype = float)
            i = i+1

        A = np.zeros(nGroups + 2)
        A[0] = verticesArray[0][0]
        A[-1] = verticesArray[-1][-1]

        for i in range(1,verticesArray.shape[0]+1):
            A[i] = verticesArray[i-1][1]

        fuzzy = np.zeros((self.data.shape[0], nGroups), dtype = float)

        j = 0
        for dado in self.data:
            fuzzyCalc = 0
            for u in range(1,A.shape[0]-1):
                calc = self.triangular(dado, A[u-1], A[u], A[u+1])
                fuzzyCalc = calc
                fuzzy[j, u-1] = calc
            j = j+1

        return fuzzy

    def pyfts(self):

      nGroups = 10
      part = Grid.GridPartitioner(data = self.data, npart = 10)

      verticesArray = np.zeros((nGroups,3), dtype = float)
      i = 0

      for key, value in part.sets.items():
        verticesStr = str(value)[str(value).find('[')+1:str(value).find(']')]
        vertices = verticesStr.split(',')
        verticesArray[i] = np.array(vertices, dtype = float)
        i = i+1

      A = np.zeros(nGroups + 2)
      A[0] = verticesArray[0][0]
      A[-1] = verticesArray[-1][-1]

      for i in range(1,verticesArray.shape[0]+1):
        A[i] = verticesArray[i-1][1]

      fuzzy = np.zeros(self.data.shape[0], dtype = int)

      j = 0
      for dado in self.data:
        fuzzyCalc = 0
        for u in range(1,A.shape[0]-1):
          calc = self.triangular(dado, A[u-1], A[u], A[u+1])
          if calc > fuzzyCalc:
            fuzzyCalc = calc
            fuzzy[j] = u - 1
        j = j+1

      return fuzzy

    def anfis(self):

        def y_hat_func(x, n, m, c, s, p, q):

            a = 0
            b = 0
            w = np.zeros(m)
            y = np.zeros(m)


            for j in range(m):
                w[j] = 1
                y[j] = q[j]
                for i in range(n):
                    w[j] = w[j]*np.exp(-0.5*((x[i]-c[i,j])/s[i,j])**2)
                    y[j] = y[j]+p[i,j]*x[i]
                a = a + w[j]*y[j]
                b = b + w[j]

            ys = a/b

            return ys, w, y, b

        m = 10
        x_train = np.array(self.data, ndmin = 2).T
        np_train = np.shape(x_train)[0]
        yd = np.array(x_train)
        n = np.shape(x_train)[1]
        alpha = 0.01
        maxEpocas = 100

        xmin = np.min(x_train, axis = 0)
        xmax = np.max(x_train, axis = 0)

        delta = (xmax - xmin)/(m-1)

        # Inicializar os parâmetros c, s, p e q

        c = np.zeros((n,m))
        s = np.zeros((n,m))

        for j in range(m):
            for i in range (n):
                c[i,j] = xmin[i] + [j-1]*delta
                s[i,j] = 0.5*delta*(1/np.sqrt(np.log(4)))

        p = np.random.rand(n,m)
        q = np.random.rand(m)

        ei2 = 0
        epoca = 0
        erroEpoca = np.ones(1)*np.inf
        xFuzzy = np.zeros((np_train,n,m))

        while erroEpoca[epoca] > 1e-1 and epoca <= maxEpocas:
            for k in range(np_train):
                xin = x_train[k,:]
                ys, w, y, b = y_hat_func(xin, n, m, c, s, p, q)
                dedys = ys - yd[k]
                for j in range(m):
                    dysdw = (y[j]-ys)/b
                    dysdy = w[j]/b
                    for i in range(n):
                        dydp = xin[i]
                        p[i,j] = p[i,j] - alpha*dydp*dysdy*dedys
                        dwdc = w[j]*((xin[i] - c[i,j])/(s[i,j]**2))
                        c[i,j] = c[i,j] - alpha*dwdc*dysdw*dedys
                        dwds = w[j]*(((xin[i] - c[i,j])**2)/(s[i,j]**3))
                        s[i,j] = s[i,j] - alpha*dwds*dysdw*dedys
                        xFuzzy[k,i,j] = np.exp(-0.5*((xin[i]-c[i,j])/s[i,j])**2)

                    dydq = 1
                    q[j] = q[j]-alpha*dedys*dysdy*dydq

                ei2 = (ys - yd[k])**2
                ei2 = ei2 + ei2

            epoca += 1
            erroEpoca = np.append(erroEpoca, ei2)

        xFTS = self.fuzzify(xFuzzy)
        return xFTS

    def nfn(self):
        def y_nfn_func(x, n, m, delta, b, xmin, xmax, w):
            k1 = np.zeros(n, dtype = int)
            muk1 = np.zeros(n)
            k2 = np.zeros(n, dtype = int)
            muk2 = np.zeros(n)
            yi = np.zeros(n)

            for i in range(n):
                if x[i] <= xmin[i]:
                    k1[i] = 0
                    muk1[i] = 1
                elif x[i] >= xmax[i]:
                    k1[i] = m - 2
                    muk1[i] = 0
                else:
                    k1[i] = int((x[i] - xmin[i])/delta[i])
                    muk1[i] = 1 - (x[i] - b[i, k1[i]])/delta[i]

                k2[i] = k1[i] + 1
                muk2[i] = 1 - muk1[i]
                yi[i] = muk1[i]*w[i,k1[i]] + muk2[i]*w[i,k2[i]]

            ys = np.sum(yi)

            return ys, yi, k1, k2, muk1, muk2

        m = 10
        x_train = np.array(self.data, ndmin = 2).T
        np_train = np.shape(x_train)[0]
        yd = np.array(x_train)
        n = np.shape(x_train)[1]
        alpha = 0.01
        maxEpocas = 1

        xmin = np.min(x_train, axis = 0)
        xmax = np.max(x_train, axis = 0)
        b = np.zeros((n,m))

        # Inicializar os parâmetros b, w e delta

        delta = (xmax - xmin)/(m-1)
        for j in range(m):
            for i in range(n):
                b[i,j] = xmin[i] + (j)*delta[i]

        w = np.random.rand(n,m)

        ei2 = 0
        epoca = 0
        erroEpoca = np.ones(1)*np.inf
        xfuzzy = np.zeros((np_train, m))

        while erroEpoca[epoca] > 1e-5 and epoca <= maxEpocas:
            for k in range(np_train):
                xin = x_train[k,:]
                ys, yi, k1, k2, muk1, muk2 = y_nfn_func(xin, n, m, delta,
                                                b, xmin, xmax, w)
                dedys = ys - yd[k]
                for i in range(n):
                    w[i,k1[i]] = w[i,k1[i]] - alpha*dedys*muk1[i]
                    w[i,k2[i]] = w[i,k2[i]] - alpha*dedys*muk2[i]
                    xfuzzy[k,k1[i]] = muk1[i]
                    xfuzzy[k,k2[i]] = muk2[i]

                ei2 = (ys - yd[k])**2
                ei2 = ei2 + ei2

            epoca += 1
            erroEpoca = np.append(erroEpoca, ei2)

        xFTS = xfuzzy
        return xFTS

    def fuzzify(self,xfuzzy):
        xFTS = np.zeros(xfuzzy.shape[0], dtype = int)
        for k in range(xfuzzy.shape[0]):
            xFTS[k] = np.where(xfuzzy[k] == np.max(xfuzzy[k], axis = 1))[1][0]

        return xFTS

    def plotError(self, erroAnfis):
        plt.plot(erroAnfis)
        plt.show()


In [ ]:
# %% Load file
# functionUCR_time_series_test

path_to_data = '/content/drive/My Drive/Doutorado/Regular/modelo/UCR/'
dataSetName = 'DodgerLoopWeekend'
totalPath = path_to_data + dataSetName + "/"
TRAIN = open(totalPath + dataSetName + '_TRAIN.ts', 'r')
TEST = open(totalPath + dataSetName + '_TEST.ts', 'r')

# %%
trainTS = {}
i = 0
isData = 0

for line in TRAIN:
  if (not(isData)):
    if (line.find('@data') >= 0):
      isData = 1
    continue
  if (isData):
    trainTS[i] = TimeSeries(line)
    i = i + 1

# %%
testTS = {}
i = 0
isData = 0

for line in TEST:
  if (not(isData)):
    if (line.find('@data') >= 0):
      isData = 1
    continue
  if (isData):
    testTS[i] = TimeSeries(line)
    i = i + 1

In [ ]:
# %%
def euclideanDist(x1,y1,x2,y2):
    dist = ((x1-x2)**2 + (y1-y2)**2)**(1/2)
    return dist


def calcDist(x1,x2):
    s = np.zeros((x1.shape[0], x2.shape[0]))
    for i in range(x1.shape[0]):
        for j in range(x2.shape[0]):
            s[i,j] = euclideanDist(x1[i],x2[i],x1[j],x2[j])
    return s

#%%
def saveArray(s, tsNumber, label, setType):
    fig = plt.figure()
    ax0 = fig.add_subplot(111)
    ax0.pcolor(s, cmap='Greys')
    ax0.axis('off')
    plt.savefig(totalPath + setType + '/' + str(label) + '/ts_'
                + str(tsNumber+1), bbox_inches='tight', pad_inches = 0)
    plt.close(fig)

def plotRecurrence(s, tsNumber, label, setType):
    fig = plt.figure()
    ax0 = fig.add_subplot(111)
    ax0.pcolor(s, cmap='Greys')

    ax0.axis('off')
    if os.path.isfile(totalPath + setType + '/' + str(label) + '/ts_' \
                + str(tsNumber)):
        pass
    else:
        plt.savefig(totalPath + setType + '/' + str(label) + '/ts_'
                    + str(tsNumber), bbox_inches='tight', pad_inches = 0)
    plt.close(fig)

def matrixImage(timeSerie):
    sortMatrix = np.sort(timeSerie)
    indexColumn = np.zeros((timeSerie.shape[0],timeSerie.shape[0]))
    for i in range(timeSerie.shape[0]):
        tsmatch = np.where(sortMatrix == timeSerie[i])[0][0]
        indexColumn[tsmatch] = 1

    return indexColumn

def polarPlot(tspolar, setType, r):
    theta = np.linspace(0, 2* np.pi, tspolar.data.shape[0])

    fig, ax0 = plt.subplots(subplot_kw={'projection': 'polar'})
    ax0.plot(theta, r, color = 'black')
    ax0.axis('off')

    path = totalPath + setType + 'New/' + str(tspolar.label)

    if os.path.isdir(path):
        plt.savefig(totalPath + setType + 'New/' + str(tspolar.label) + '/ts_'
                    + str(i+1), bbox_inches='tight', pad_inches = 0)
    else:
        os.makedirs(path)
        plt.savefig(totalPath + setType + 'New/' + str(tspolar.label) + '/ts_'
                    + str(i+1), bbox_inches='tight', pad_inches = 0)

    plt.close(fig)

In [ ]:
# %% Criando imagens de treinamento
imageType = 'Polar'
partType = 'Grid'
ftsType = 'PYFTS'
if imageType == 'WFTS':
    setType = 'trainData' + imageType + '_' + ftsType + partType
else:
    setType = 'trainData' + imageType

print('Imagens de treinamento ' + imageType + ' da base ' + dataSetName + ':\n')
rootPath = totalPath + setType
if os.path.isdir(rootPath):
    shutil.rmtree(rootPath)

for i in range(len(trainTS)):
    print('Criando imagem de treinamento ' + str(i+1) + ' / ' + str(len(trainTS)))
    if trainTS[i].hasNA:
        print('Has NA')
        continue
    if imageType == 'WFTS':
        if ftsType == 'PYFTS':
            ts = trainTS[i].wpyfts(partType).transpose()
        elif ftsType == 'NFN':
            ts = trainTS[i].nfn().transpose()
    elif imageType == 'Polar':
        ts = trainTS[i].data
    x1 = ts[0:-1]
    x2 = ts[1:]
    s = ts
    path = totalPath + setType + '/' + str(trainTS[i].label)
    if os.path.isdir(path):
        if imageType == 'WFTS':
            saveArray(s, i, trainTS[i].label, setType)
        elif imageType == 'Polar':
            trainTS[i].image = polarPlot(trainTS[i], setType, ts)
        else:
            print('Erro! Nenhum tipo de imagem válido foi definido!')
            break

    else:
        os.makedirs(path)
        if imageType == 'WFTS':
            saveArray(s, i, trainTS[i].label, setType)
        elif imageType == 'Polar':
            trainTS[i].image = polarPlot(trainTS[i], setType, ts)
        else:
            print('Erro! Nenhum tipo de imagem válido foi definido!')
            break

Imagens de treinamento Polar da base DodgerLoopWeekend:

Criando imagem de treinamento 1 / 20
Criando imagem de treinamento 2 / 20
Criando imagem de treinamento 3 / 20
Criando imagem de treinamento 4 / 20
Criando imagem de treinamento 5 / 20
Criando imagem de treinamento 6 / 20
Criando imagem de treinamento 7 / 20
Criando imagem de treinamento 8 / 20
Has NA
Criando imagem de treinamento 9 / 20
Criando imagem de treinamento 10 / 20
Has NA
Criando imagem de treinamento 11 / 20
Criando imagem de treinamento 12 / 20
Criando imagem de treinamento 13 / 20
Criando imagem de treinamento 14 / 20
Criando imagem de treinamento 15 / 20
Criando imagem de treinamento 16 / 20
Criando imagem de treinamento 17 / 20
Criando imagem de treinamento 18 / 20
Criando imagem de treinamento 19 / 20
Criando imagem de treinamento 20 / 20


In [ ]:
# %% Criando imagens de teste
if imageType == 'WFTS':
    setType = 'testData' + imageType + '_' + ftsType + partType
else:
    setType = 'testData' + imageType

print('\nImagens de teste ' + imageType + ' da base ' + dataSetName + ':\n')
rootPath = totalPath + setType
if os.path.isdir(rootPath):
    shutil.rmtree(rootPath)

for i in range(len(testTS)):
    print('Criando imagem de teste ' + str(i+1) + ' / ' +str(len(testTS)))
    if testTS[i].hasNA:
        print('Has NA')
        continue
    if imageType == 'WFTS':
        if ftsType == 'PYFTS':
            ts = testTS[i].wpyfts(partType).transpose()
        elif ftsType == 'NFN':
            ts = testTS[i].nfn().transpose()
    elif imageType == 'Polar':
        ts = testTS[i].data
    else:
        print('Erro! Nenhum tipo de imagem válido foi definido!')
        break

    x1 = ts[0:-1]
    x2 = ts[1:]
    s = ts
    path = totalPath + setType + '/' + str(testTS[i].label)
    if os.path.isdir(path):
        if imageType == 'WFTS':
            saveArray(s, i, testTS[i].label, setType)
        elif imageType == 'Polar':
            if testTS[i].hasNA:
                continue
            testTS[i].image = polarPlot(testTS[i], setType, ts)
        else:
            print('Erro! Nenhum tipo de imagem válido foi definido!')
            break

    else:
        os.makedirs(path)
        if imageType == 'WFTS':
            saveArray(s, i, testTS[i].label, setType)
        elif imageType == 'Polar':
            if testTS[i].hasNA:
                continue
            testTS[i].image = polarPlot(testTS[i], setType, ts)
        else:
            print('Erro! Nenhum tipo de imagem válido foi definido!')
            break


Imagens de teste Polar da base DodgerLoopGame:

Criando imagem de teste 1 / 138
Criando imagem de teste 2 / 138
Criando imagem de teste 3 / 138
Criando imagem de teste 4 / 138
Criando imagem de teste 5 / 138
Criando imagem de teste 6 / 138
Criando imagem de teste 7 / 138
Criando imagem de teste 8 / 138
Has NA
Criando imagem de teste 9 / 138
Criando imagem de teste 10 / 138
Criando imagem de teste 11 / 138
Criando imagem de teste 12 / 138
Criando imagem de teste 13 / 138
Criando imagem de teste 14 / 138
Has NA
Criando imagem de teste 15 / 138
Criando imagem de teste 16 / 138
Criando imagem de teste 17 / 138
Criando imagem de teste 18 / 138
Criando imagem de teste 19 / 138
Criando imagem de teste 20 / 138
Criando imagem de teste 21 / 138
Criando imagem de teste 22 / 138
Has NA
Criando imagem de teste 23 / 138
Has NA
Criando imagem de teste 24 / 138
Criando imagem de teste 25 / 138
Criando imagem de teste 26 / 138
Criando imagem de teste 27 / 138
Criando imagem de teste 28 / 138
Criando 